In [63]:
from urllib.request import Request, urlopen
import requests
from bs4 import BeautifulSoup as soup
import pandas as pd
from time import sleep

In [2]:
# URL base de la página que vamos a analizar
base_url = "https://www.logrolimpio.com/catalogo-segunda-vida/?listing-page="

In [75]:
#Creación de la función que extrae el contenido de la web
def extractSoup(url):
    req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    page_soup = soup(webpage, "html.parser")
    return page_soup

def bot_send_text(bot_message):
    bot_token = "7517642163:AAH-QbrmfUYLljOzpDIbVKZFd8wxIN_ffUg"
    bot_chatID = "5544442" #Privado
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&text=' + bot_message
    response = requests.get(send_text)
    return response

def report(producto, categoria, link):
    mensaje = 'Se ha encontrado el producto ' + producto + ' que pertenece a la categoría ' + categoria + '. \n\n' + link
    bot_send_text(mensaje)


In [99]:
productos = []
ids = []
categorias = []
links = []
images = []

In [4]:
pagina = 1
url = f"{base_url}{pagina}"
page_soup = extractSoup(url)

In [5]:
#Generar lista con los productos de la página
for producto in page_soup.findAll('h3'):
    productos.append(producto.text)

In [ ]:
#Extracción de las ids de los productos
for producto in productos:
    ids.append(producto[-10:])

In [ ]:
#Extracción de las categorías de los productos
categorias = page_soup.findAll('li', {"class":"rt-categories"})
for i in range(len(categorias)):
    categorias[i] = categorias[i].get_text(strip=True)

In [6]:
for i in page_soup.findAll('h3'):
    for j in i.find_all('a'):
        link = j.get('href')
        links.append(link)

In [67]:
for listing in page_soup.select('.listing-item .listing-thumb img'):
    img_url = listing.get('src')
    if img_url:
        images.append(img_url)

In [78]:
for i in range(len(productos)):
    producto = productos[i]
    categoria = categorias[i]
    link = links[i]
    imagen = images[i]
    report(producto, categoria, link)
    sleep(20)

In [77]:
report(producto, categoria, link)

In [98]:
df = pd.DataFrame({'id': ids,
                   'producto': productos,
                   'categoria': categorias,
                   'link': links,
                   'imagen': images},
                   columns=['id', 'producto', 'categoria', 'link', 'imagen'])

# Guardamos los datos en un archivo CSV
df.to_csv('productos.csv', index=False, encoding='utf-8-sig', sep=';')

In [48]:
type(mensaje)

str

In [97]:
ids = []
for producto in productos:
    ids.append(producto[-10:])

In [59]:
producto = f'Se ha encontrado el producto {productos[1]}'
print(producto)
type(producto)

Se ha encontrado el producto Maleta de viaje *240830_055


str

In [3]:
#bucle para recorrer todas las páginas

pagina = 1
while productos != [] or pagina < 3:
#while pagina < 10:
    url = f"{base_url}{pagina}"
    req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    page_soup = soup(webpage, "html.parser")
    #title = page_soup.find("title")
    productos_bs4 = page_soup.findAll("h3")
    for producto in productos_bs4:
        print(producto)
    break
    #pagina +=1

<h3 class="listing-title rtcl-listing-title"><a href="https://www.logrolimpio.com/catalogo/planchas-240830_010/">Planchas *240830_010</a></h3>
<h3 class="listing-title rtcl-listing-title"><a href="https://www.logrolimpio.com/catalogo/libros-peter-rabbit-240830_009/">Libros Peter Rabbit *240830_009</a></h3>
<h3 class="listing-title rtcl-listing-title"><a href="https://www.logrolimpio.com/catalogo/tensiometro-240830_008/">Tensiómetro *240830_008</a></h3>
<h3 class="listing-title rtcl-listing-title"><a href="https://www.logrolimpio.com/catalogo/libros-historia-de-la-rioja-240830_007/">Libros Historia de la Rioja *240830_007</a></h3>
<h3 class="listing-title rtcl-listing-title"><a href="https://www.logrolimpio.com/catalogo/platos-llanos-grandes-medianos-y-pequenos-240830_006/">Platos llanos grandes, medianos y pequeños *240830_006</a></h3>
<h3 class="listing-title rtcl-listing-title"><a href="https://www.logrolimpio.com/catalogo/libros-plantas-setas-y-hongos-de-la-rioja-240830_005/">Libros